In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade transformers

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd
data=pd.read_csv("/content/comp1804_coursework_dataset_23-24.csv")

In [ ]:
duplicates_rows=data.duplicated()
print(duplicates_rows.sum())

215


In [ ]:
#deleting duplicate data
new_data=data.drop_duplicates()
duplicates_rows=new_data.duplicated()
print(duplicates_rows.sum())


0


In [ ]:
# GROUPING THEM ACCORDING TO FEATURE
a1=new_data['category'].value_counts().index
for i in range(0,9):
 new_data.loc[new_data['category']==a1[i],'category'] = a1[i].capitalize()
new_data['category'].value_counts()

,count
category,
Biographies,2899
Philosophy,2524
Programming,1948
Artificial intelligence,1537
Movies about artificial intelligence,163


In [ ]:
new_data=new_data.dropna(subset=['category'])
new_data.isna().sum()

,0
par_id,0
paragraph,0
has_entity,0
lexicon_count,0
difficult_words,18
last_editor_gender,0
category,0
text_clarity,9062


In [ ]:
import pandas as pd  # For label encoding
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from datasets import Dataset
import pandas as pd  # For label encoding
# Import the standard Adam optimizer
from torch.optim import Adam

# === 1. Prepare data ===
X = new_data['paragraph'][:100].astype(str).tolist()
y = new_data['category'][:100].astype(str).tolist()

# Label‐encode categories
label_mapping = {label: idx for idx, label in enumerate(pd.unique(y))}
y_encoded = [label_mapping[label] for label in y]

# === 2. Tokenizer & Dataset ===
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=256)

dataset = Dataset.from_dict({"text": X, "label": y_encoded})
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# PyTorch format
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# Train/test split
split_idx = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(split_idx))
test_dataset  = tokenized_dataset.select(range(split_idx, len(tokenized_dataset)))

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)
test_loader  = DataLoader(test_dataset, batch_size=32)



# === 3. Model, optimizer, scheduler ===
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_mapping)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Use the standard Adam optimizer instead of AdamW
optimizer = Adam(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# === 4. Training Loop ===
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} — avg loss: {avg_loss:.4f}")

# === 5. Evaluation ===
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

accuracy = correct / total * 100
print(f"Test set accuracy: {accuracy:.2f}%")


<ipython-input-10-bd2f4759f7da>:15: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  label_mapping = {label: idx for idx, label in enumerate(pd.unique(y))}
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 — avg loss: 1.4555
Epoch 2/3 — avg loss: 1.0740
Epoch 3/3 — avg loss: 0.8725
Test set accuracy: 60.00%
